In [1]:
import polars as pl
## Make polars output wide enough for links and more text
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [2]:
## Load the data
df = pl.read_parquet("../reddit_data/reddit.parquet")

In [3]:
df.shape

(5528298, 14)

# Exploring the data
## Non-Unique Values
Some observations about the values in in the `reddit_text` column of the dataset:

* There are 38924 non-unique values that appear in a total of 306,242 rows
* There are 5222056 rows with unique values (~94.46\% of all rows)
* Emojis apear frequenlty, and BERT models treat emojis as "unknown" tokens
* Many many non-unique entries appear to be some flavor of "Yes", "No", or "Thank you".

Of the non-unique values:

* The values `""`, `"[deleted]"`, and `"[removed]"` make up a total of 77,914 rows (a little over 25\% of all non-unique values)
    * My initial impression is that rows with values of `"[deleted]"`, and `"[removed]"` can safely be removed from the dataset.
    * The value `""` only appears as a *comment* 6 times. In all 42340 other instances it is a *submission*.
        * Manual inspection of the 6 *comment* instances seems to indicate that these posts we edited to be empty by the author, but not deleted. They can be safely dropped.
        * A cursory inspection of a sample of the *submission* instances seems to indicate that many of these fall into one of these situations:
            * The submission title is the question or statement to which comments that follow are responding to
            * The submission is an image with no text (often still with some information in the title).
        * My initial impression was that we could either just delete these posts, or replace their `reddit_text` value with the value in `reddit_title`
        * These values only make up ~0.77\% of all values in the `reddit_text` column, but in a few subreddits they make up a larger proprtion:
            * In `FedEmployees` 107 of the total 280 posts (~38\%)
            * In  `disney`: 2555 of 43954 (~5.8\%)
            * In `RiteAid`: 170 of 3970 (~4.3%)
            * In `PaneraEmployees`: 115 of 2694 (4.3\%)
            * In `WalmartEmployees`: 285 of 10752 (2.7\%)
            * In all others, <2\%.
* Probably about 10,000 additional posts can be attributed to bots and can likely be safely removed.

In [4]:
## Count the frequency of values in the 'reddit_text' column.
value_counts = df["reddit_text"].value_counts().sort(
    by="count", descending=True)

In [5]:
## Observing how many times each non-unique value appears
max_rows = 10
pl.Config.set_tbl_rows(max_rows)
start_at=0
print(value_counts[start_at:start_at+max_rows])
pl.Config.set_tbl_rows(10)

shape: (10, 2)
┌─────────────┬───────┐
│ reddit_text ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
│             ┆ 42346 │
│ [deleted]   ┆ 23244 │
│ [removed]   ┆ 12324 │
│ Yes         ┆ 3611  │
│ Thank you!  ┆ 3564  │
│ No          ┆ 2617  │
│ Thank you   ┆ 1701  │
│ Thanks!     ┆ 1698  │
│ Lol         ┆ 1672  │
│ Same        ┆ 1444  │
└─────────────┴───────┘


polars.config.Config

In [6]:
## Count the frequency of the frequency of values in the 'reddit_text' column.
value_value_counts=value_counts.with_columns(
    val_counts=value_counts["count"]
).drop("count")["val_counts"].value_counts().sort(
    by="count", descending=False)

pl.Config.set_tbl_rows(6)
print(value_value_counts)
pl.Config.set_tbl_rows(10)

shape: (240, 2)
┌────────────┬─────────┐
│ val_counts ┆ count   │
│ ---        ┆ ---     │
│ u32        ┆ u32     │
╞════════════╪═════════╡
│ 42346      ┆ 1       │
│ 23244      ┆ 1       │
│ 12324      ┆ 1       │
│ …          ┆ …       │
│ 3          ┆ 5842    │
│ 2          ┆ 23330   │
│ 1          ┆ 5222056 │
└────────────┴─────────┘


polars.config.Config

In [7]:
## Create a df with only the values that are not unique
non_uniuqes = value_counts[0:38921]
non_uniuqes.write_parquet("../temp_data/non_unique_text_values.parquet")

In [8]:
## Check out how many `""` values apear as submissions
df.filter(pl.col("reddit_text")=="").filter(pl.col("aware_post_type")=="submission").shape

(42340, 14)

In [9]:
## Check out how many `""` values apear as comments
df.filter(pl.col("reddit_text")=="").filter(pl.col("aware_post_type")!="submission").shape

(6, 14)

In [10]:
## View the comments with `""` as their 'reddit_text'
df.filter(pl.col("reddit_text")=="").filter(pl.col("aware_post_type")!="submission")

aware_post_type,aware_created_ts,reddit_id,reddit_name,reddit_created_utc,reddit_author,reddit_text,reddit_permalink,reddit_title,reddit_url,reddit_subreddit,reddit_link_id,reddit_parent_id,reddit_submission
str,str,str,str,i64,str,str,str,str,str,str,str,str,str
"""comment""","""2023-05-25T05:27:01""","""jljk39m""","""t1_jljk39m""",1685006821,"""Designer-Seesaw1381""","""""","""/r/nursing/comments/13r42vt/not_like_other_nurses/jljk39m/""",null,null,"""nursing""","""t3_13r42vt""","""t1_jlirq3l""","""13r42vt"""
"""comment""","""2023-05-25T05:29:13""","""jljk8zx""","""t1_jljk8zx""",1685006953,"""Designer-Seesaw1381""","""""","""/r/nursing/comments/13r42vt/not_like_other_nurses/jljk8zx/""",null,null,"""nursing""","""t3_13r42vt""","""t1_jlis8vs""","""13r42vt"""
"""comment""","""2023-07-30T18:54:45""","""ju4rte7""","""t1_ju4rte7""",1690757685,"""malphasia""","""""","""/r/Target/comments/15dttfc/i_hate_how_our_discount_works_its_absolute_garbage/ju4rte7/""",null,null,"""Target""","""t3_15dttfc""","""t1_ju4oi61""","""15dttfc"""
"""comment""","""2023-08-30T17:29:12""","""jyfmhiz""","""t1_jyfmhiz""",1693430952,"""IntentOnChivalry""","""""","""/r/fidelityinvestments/comments/165nt3o/fidelity_service_is_unbelievable/jyfmhiz/""",null,null,"""fidelityinvestments""","""t3_165nt3o""","""t1_jyf1mxb""","""165nt3o"""
"""comment""","""2023-10-12T16:31:37""","""k4m2ci4""","""t1_k4m2ci4""",1697142697,"""JustHereToLearn13""","""""","""/r/sysadmin/comments/176ew3e/tag_printer_issue_need_help/k4m2ci4/""",null,null,"""sysadmin""","""t3_176ew3e""","""t1_k4lu39g""","""176ew3e"""
"""comment""","""2024-01-15T11:29:57""","""khzfp4f""","""t1_khzfp4f""",1705336197,"""ainyg6767""","""""","""/r/WaltDisneyWorld/comments/1970i8l/weekly_faqs_general_discussion_thread/khzfp4f/""",null,null,"""WaltDisneyWorld""","""t3_1970i8l""","""t3_1970i8l""","""1970i8l"""


In [11]:
## Look at how many rows are associated with each subreddit
pl.Config.set_tbl_rows(34)
print(df["reddit_subreddit"].value_counts().sort(
    by="count", descending=True))
pl.Config.set_tbl_rows(10)

shape: (34, 2)
┌─────────────────────┬────────┐
│ reddit_subreddit    ┆ count  │
│ ---                 ┆ ---    │
│ str                 ┆ u32    │
╞═════════════════════╪════════╡
│ nursing             ┆ 789499 │
│ walmart             ┆ 630962 │
│ sysadmin            ┆ 557558 │
│ starbucks           ┆ 393597 │
│ WaltDisneyWorld     ┆ 373138 │
│ Target              ┆ 340401 │
│ UPSers              ┆ 262483 │
│ Disneyland          ┆ 231981 │
│ Lowes               ┆ 198805 │
│ CVS                 ┆ 179598 │
│ McDonaldsEmployees  ┆ 174679 │
│ cybersecurity       ┆ 161868 │
│ Fedexers            ┆ 154572 │
│ GameStop            ┆ 137071 │
│ starbucksbaristas   ┆ 132019 │
│ fidelityinvestments ┆ 129423 │
│ Bestbuy             ┆ 121077 │
│ wholefoods          ┆ 82052  │
│ Panera              ┆ 79436  │
│ DisneyWorld         ┆ 65549  │
│ DollarTree          ┆ 59745  │
│ TjMaxx              ┆ 46286  │
│ disney              ┆ 43954  │
│ McLounge            ┆ 38627  │
│ GeneralMotors       ┆ 3727

polars.config.Config

In [12]:
## Create a datframe with just the submissions with '""' as their 'reddit_text'
blanks = df.filter(
    pl.col("reddit_text")=="").filter(
    pl.col("aware_post_type")=="submission")[['reddit_subreddit',
                                              'reddit_name',
                                              'reddit_created_utc',
                                              'reddit_permalink',
                                              'reddit_text',
                                              'reddit_title']]

## Count how many blank texts are in each subreddit (sort by subreddit)
blank_count = blanks["reddit_subreddit"].value_counts().sort(by="reddit_subreddit")
## Count how many total texts are in each subreddit (sort by subreddit)
total_count = df["reddit_subreddit"].value_counts().sort(by="reddit_subreddit")

## Append the total text count to the blank text count
blank_count = blank_count.with_columns(total_count=total_count["count"])

## Calculuate the percent of texts in each subreddit that are blank
## and print sorted by that percentage
pl.Config.set_tbl_rows(34)
print(blank_count.with_columns(
    pct=blank_count["count"]/blank_count["total_count"]
    ).sort(by="pct", descending=True))
pl.Config.set_tbl_rows(10)

shape: (34, 4)
┌─────────────────────┬───────┬─────────────┬──────────┐
│ reddit_subreddit    ┆ count ┆ total_count ┆ pct      │
│ ---                 ┆ ---   ┆ ---         ┆ ---      │
│ str                 ┆ u32   ┆ u32         ┆ f64      │
╞═════════════════════╪═══════╪═════════════╪══════════╡
│ FedEmployees        ┆ 107   ┆ 280         ┆ 0.382143 │
│ disney              ┆ 2555  ┆ 43954       ┆ 0.058129 │
│ RiteAid             ┆ 170   ┆ 3970        ┆ 0.042821 │
│ PaneraEmployees     ┆ 115   ┆ 2694        ┆ 0.042687 │
│ WalmartEmployees    ┆ 285   ┆ 10752       ┆ 0.026507 │
│ BestBuyWorkers      ┆ 104   ┆ 5629        ┆ 0.018476 │
│ McLounge            ┆ 650   ┆ 38627       ┆ 0.016828 │
│ DisneyWorld         ┆ 971   ┆ 65549       ┆ 0.014813 │
│ walmart             ┆ 8078  ┆ 630962      ┆ 0.012803 │
│ TjMaxx              ┆ 530   ┆ 46286       ┆ 0.011451 │
│ CVS                 ┆ 2045  ┆ 179598      ┆ 0.011387 │
│ cybersecurity       ┆ 1837  ┆ 161868      ┆ 0.011349 │
│ Panera        

polars.config.Config

In [13]:
## Add a column that holds the number of characters in the non-unique texts
non_uniuqes_lens = non_uniuqes.with_columns(
    text_len = non_uniuqes["reddit_text"].str.len_bytes()
)

In [14]:
## Trying to identify likely bot posts
likely_bots = non_uniuqes_lens.filter(
    non_uniuqes_lens["text_len"]>33).filter(
        pl.col("count")>20
    ).sort(by="text_len", descending=True)

pl.Config.set_fmt_str_lengths(5000)
pl.Config.set_tbl_rows(75)
likely_bots

reddit_text,count,text_len
str,u32,u32
"""I'm going to point you to the usual resources I use for newer folks: 1. [The forum FAQ](https://reddit.com/r/cybersecurity/w/faq) 2. [This blog post on getting started](https://bytebreach.com/?p=72) 3. [This blog post on other/alternative resources](https://bytebreach.com/?p=95) 4. [These links to career roadmaps](https://www.reddit.com/r/cybersecurity/comments/smbnzt/mentorship_monday/hw8mw4k/) 5. [These training/certification roadmaps](https://www.reddit.com/r/cybersecurity/comments/sgmqxv/mentorship_monday/hv7ixno/) 6. [These links on learning about the industry](https://www.reddit.com/r/cybersecurity/comments/sb7ugv/mentorship_monday/hux2869/) 7. [This list of InfoSec projects to pad an entry-level resume](https://www.reddit.com/r/cybersecurity/comments/sxir9c/as_a_entry_level_professional_trying_to_get_into/hxsm5qn/) 8. [This extended mentorship FAQ](https://bytebreach.com/mentorship/) 9. [These links for interview prep](https://old.reddit.com/r/cybersecurity/comments/ybwsz9/mentorship_monday_post_all_career_education_and/itqbzq4/) Early on, you're going to want to learn more about the industry in order to help inform your decision about whether or not InfoSec is for you; such knowledge will also help guide your initial career trajectory based on what roles/responsibilities look attractive. (see links 3, 4, and 6). If you think that you do want to pursue a career, then you'll want to buoy your knowledge base with understanding IT/CS fundamentals more broadly. Some people pursue degrees, as an example ([although this is certainly not the only approach worth considering](https://bytebreach.com/?p=142)). (see links 1, 2, and 5). Eventually you'll need to work on improving your employability. This manifests in a variety of ways, but the most notable is probably accumulating [relevant industry-recognized certifications](https://bytebreach.com/?p=152). (see links 5 and 7) Other actions to improve your employability may include: * [Continue to leverage free resources to hone your craft or acquire new skills](https://bytebreach.com/hacking-helpers-learn-cybersecurity/). * [Pursue in-demand certifications to improve your employability](https://www.reddit.com/r/cybersecurity/comments/sgmqxv/mentorship_monday/hv7ixno/). * [Vie for top placement in competitive CTF competitions](https://ctftime.org/). * Foster a professional network via [jobs listings sites](https://www.weidert.com/blog/best-ways-to-gain-more-connections-on-linkedin) and [in-person conferences](https://infosec-conferences.com/). * Continue the job hunt for relevant experience and [take note of the feedback you receive in interviews](https://www.reddit.com/r/cybersecurity/comments/vg864z/mentorship_monday_post_all_career_education_and/id2tsr3/); consider expanding the aperture of jobs considered to include cyber-adjacent lines of work (software dev, systems administration, etc.) - this is a channel for you to build relevant years of experience. * Consider pursuing a degree-granting program (and internship experience while holding a student status). * [Post your resume to this thread for constructive feedback](https://bytebreach.com/how-to-write-an-infosec-resume/). * [Apply your skills into some projects in order to demonstrate your expertise](https://www.reddit.com/r/cybersecurity/comments/sxir9c/as_a_entry_level_professional_trying_to_get_into/hxsm5qn/).""",24,3380
"""Please post all your **general** WDW comments and **FAQs** here, as well as any **reopening-related** questions, discussion, etc. **Examples might include things like:** * What should I do to **prepare** for **the weather (heat, rain, tropical storm, etc.)** during my upcoming trip? * How do I get **tickets** for an **after-hours event,** such as **Mickey’s Not-So-Scary Halloween Party (MNSSHP), Jollywood Nights,** or **Mickey’s Very Merry Christmas Party (MVMCP)**? What happens if they’re **sold out** on the night we want to attend? * How does the **TRON/Guardi

In [15]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(10)
likely_bots["count"].sum()

10346